In [1]:
# ======================================================================================
# Libraries
# ======================================================================================
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split #separate into train test data
from sklearn.preprocessing import OneHotEncoder
import tensorflow
from keras import models
from keras import layers
print("Load completed")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Load completed


In [2]:
# ======================================================================================
# Get DATA & split data in train/valid
# ======================================================================================
#Read the datas from csv files
train = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv', index_col=0)

#Separate train data into X and y
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

#Create a encoder
encoder = OneHotEncoder(sparse=False)

#Encode the sex with using OneHot Encoder
sex_data = encoder.fit_transform(X['Sex'].values.reshape(-1,1))
sex_df = pd.DataFrame(sex_data, columns=['Sex_F','Sex_I','Sex_M'])
sex_df = sex_df.set_index(X.index)

#Change the sex column on train data
X.drop('Sex',inplace=True,axis=1)
X = pd.concat([sex_df,X],axis=1)

# #Separate X and y into train test splits
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
# ======================================================================================
# Create Engineering Features
# ======================================================================================
import math

test ["volume"] = test["Height"] * test["Diameter"] * test["Length"]
test ["dim1"] = test["Height"] * test["Diameter"] 
test ["dim2"] = test["Height"] * test["Length"] 
test ["dim3"] = test["Diameter"] * test["Length"]
test ["total_weight"] = test["Shell Weight"] + test["Viscera Weight"] + test["Shucked Weight"]
test ["weight_volume_ratio"] = test["Weight"] / (test["Diameter"] + 1e-8 )
test ["shell_to_total_weight"] = test["Shell Weight"] / test["Weight"]
test ["viscera_to_total_weight"] = test["Viscera Weight"] / test["Weight"]
test ["shucked_to_total_weight"] = test["Shucked Weight"] / test["Weight"]

# Volume: Since crabs have a three-dimensional shape, you can calculate their volume using the length, diameter, and height features.
test["volumeV2"] = math.pi*(test["Diameter"] / 2) ** 2 *  test["Height"]
test["surface"] = 2 * math.pi * (test["Diameter"] / 2) *  (test["Height"] + (test["Diameter"] / 2))
# BMI (Body Mass Index): BMI is a measure of body fat based on the weight and height of an individual.
test["bmi"] = test["Weight"]/(test["Length"] ** 2)                                                    
# Square and Cube: Squaring or cubing a feature can capture non-linear relationships in the data.
#     df["Length Squared"] = df["Length"] ** 2
#     df["Diameter Cubed"] = df["Diameter"] ** 3
# Logarithm: Taking the logarithm of a feature can help normalize its distribution and handle extreme values:
test["log_weight"] = np.log(test["Weight"])


# Additional features
# Shell Weight Squared: You can create a new feature by squaring the shell weight. 
# This can capture any non-linear relationship between the shell weight and the target variable:
test["shell_weight_squared"] = test["Shell Weight"] * test["Shell Weight"] 

# Shell Weight Cubed: Similarly, you can create a new feature by cubing the shell weight. 
# This can capture further non-linear relationships:
test["shell_weight_cubed"] = test["Shell Weight"] * test["Shell Weight"] * test["Shell Weight"] 

test = add_pca_features(test)
test = pd.get_dummies(test)



In [4]:
train ["volume"] = train["Height"] * train["Diameter"] * train["Length"]
train ["dim1"] = train["Height"] * train["Diameter"] 
train ["dim2"] = train["Height"] * train["Length"] 
train ["dim3"] = train["Diameter"] * train["Length"]
train ["total_weight"] = train["Shell Weight"] + train["Viscera Weight"] + train["Shucked Weight"]
train ["weight_volume_ratio"] = train["Weight"] / (train["Diameter"] + 1e-8 )
train ["shell_to_total_weight"] = train["Shell Weight"] / train["Weight"]
train ["viscera_to_total_weight"] = train["Viscera Weight"] / train["Weight"]
train ["shucked_to_total_weight"] = train["Shucked Weight"] / train["Weight"]

# Volume: Since crabs have a three-dimensional shape, you can calculate their volume using the length, diameter, and height features.
train["volumeV2"] = math.pi*(train["Diameter"] / 2) ** 2 *  train["Height"]
train["surface"] = 2 * math.pi * (train["Diameter"] / 2) *  (train["Height"] + (train["Diameter"] / 2))
# BMI (Body Mass Index): BMI is a measure of body fat based on the weight and height of an individual.
train["bmi"] = train["Weight"]/(train["Length"] ** 2)                                                    
# Square and Cube: Squaring or cubing a feature can capture non-linear relationships in the data.
#     df["Length Squared"] = df["Length"] ** 2
#     df["Diameter Cubed"] = df["Diameter"] ** 3
# Logarithm: Taking the logarithm of a feature can help normalize its distribution and handle extreme values:
train["log_weight"] = np.log(train["Weight"])


# Additional features
# Shell Weight Squared: You can create a new feature by squaring the shell weight. 
# This can capture any non-linear relationship between the shell weight and the target variable:
train["shell_weight_squared"] = train["Shell Weight"] * train["Shell Weight"] 

# Shell Weight Cubed: Similarly, you can create a new feature by cubing the shell weight. 
# This can capture further non-linear relationships:
train["shell_weight_cubed"] = train["Shell Weight"] * train["Shell Weight"] * train["Shell Weight"] 

train = add_pca_features(train)
train = pd.get_dummies(train)


In [5]:
# ======================================================================================
# Create the deep learning LSTM model
# ======================================================================================
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [6]:
X_train

,Sex_F,Sex_I,Sex_M,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
id,,,,,,,,,,
20612,0.0,0.0,1.0,1.6125,1.1500,0.4000,31.269499,15.252031,6.265239,7.059025
2935,1.0,0.0,0.0,1.5250,1.1750,0.4125,30.220567,12.232809,7.002326,8.930093
73847,0.0,0.0,1.0,1.5000,1.1625,0.3875,29.114936,13.125818,6.166016,8.079607
46445,0.0,0.0,1.0,1.1250,0.8875,0.2375,12.998246,5.499803,2.891649,3.685435
66755,0.0,1.0,0.0,0.8625,0.6375,0.1875,4.280774,1.644271,0.737087,1.275727
...,...,...,...,...,...,...,...,...,...,...
37194,0.0,1.0,0.0,0.7000,0.5250,0.2000,2.041164,0.935534,0.510291,0.765436
6265,1.0,0.0,0.0,1.6500,1.2750,0.5250,39.859397,17.605040,8.093782,11.623295
54886,0.0,0.0,1.0,1.6750,1.3500,0.4375,42.013959,20.950281,8.292229,10.347568


In [7]:
# Import des librairies utiles

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, TimeDistributed, RepeatVector, Flatten
from keras.callbacks import EarlyStopping
from math import ceil

# Initialising the RNN
model = Sequential()
model.add(layers.Dense(120,activation='relu', input_shape=[10]))   #input_shape=[10]             
model.add(Dropout(0.2))
model.add(layers.Dense(240,activation='relu'))     
model.add(Dropout(0.2))
model.add(layers.Dense(240,activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(240,activation='relu'))  
model.add(Dropout(0.2))
model.add(layers.Dense(120,activation='relu'))  
model.add(Dense(1))



In [8]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
# Moniteur pour arreter le training 


In [9]:
# ======================================================================================
# FIT THE MODEL
# ======================================================================================
model.fit(X_train, y_train,
          batch_size=8, 
          epochs=5,
          validation_data=(X_valid, y_valid))

Epoch 1/5
7405/7405 [==============================] - 24s 3ms/step - loss: 7.5162 - accuracy: 1.6880e-04 - val_loss: 10.1049 - val_accuracy: 4.0510e-04
Epoch 2/5
7405/7405 [==============================] - 23s 3ms/step - loss: 5.2632 - accuracy: 1.6880e-04 - val_loss: 11.4792 - val_accuracy: 4.0510e-04
Epoch 3/5
7405/7405 [==============================] - 23s 3ms/step - loss: 4.9989 - accuracy: 1.6880e-04 - val_loss: 8.4179 - val_accuracy: 4.0510e-04
Epoch 4/5
7405/7405 [==============================] - 23s 3ms/step - loss: 4.8359 - accuracy: 1.6880e-04 - val_loss: 7.5608 - val_accuracy: 4.0510e-04
Epoch 5/5
7405/7405 [==============================] - 23s 3ms/step - loss: 4.7672 - accuracy: 1.6880e-04 - val_loss: 7.0561 - val_accuracy: 4.0510e-04


In [10]:
#Encode the sex with using OneHot Encoder
sex_data_test = encoder.fit_transform(test['Sex'].values.reshape(-1,1))
sex_df_test = pd.DataFrame(sex_data_test, columns=['Sex_F','Sex_I','Sex_M'])
sex_df_test = sex_df_test.set_index(test.index)

#Change the sex column on train data
test.drop('Sex',inplace=True,axis=1)
test = pd.concat([sex_df_test,test],axis=1)
test.head()

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Sex_F,Sex_I,Sex_M,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
id,,,,,,,,,,
74051,0.0,1.0,0.0,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
74052,0.0,1.0,0.0,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
74053,1.0,0.0,0.0,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
74054,1.0,0.0,0.0,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
74055,0.0,1.0,0.0,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [11]:
# ======================================================================================
# PREDICT
# ======================================================================================
#predit sumbission result
age = model.predict(test)

1543/1543 [==============================] - 3s 2ms/step


In [12]:
# ======================================================================================
# SUBMISSION
# ======================================================================================
#create a submission df and load to submit a result
submission = pd.DataFrame(age, index=test.index, columns=['Age'])

#Load submission to csv
submission.to_csv('/kaggle/working/submission.csv')